In [1]:
import requests
import asyncio
import aiohttp
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd
import datetime
import time
import os
import csv
import pickle
from IPython.display import clear_output # Cell output clear를 위함

In [5]:
# df_article_list_gen = pd.read_csv('seoulK.csv', encoding='utf-8', chunksize=20000, dtype={'종목코드': object})

In [6]:
def save_to_csv_one_article(one_dict, header_list):
    if os.path.isfile('seoulK_with_body.csv'): # 파일이 있는지 확인
        open_param = 'a' # 있으면 수정(뒤에 내용 추가) 모드
    else:
        open_param = 'w' # 없으면 파일 생성함
            
    with open('seoulK_with_body.csv', open_param, encoding='utf-8', newline='') as f:
        wr = csv.writer(f)
        if open_param == 'w':
            wr.writerow(header_list)
            
        writerow_pa_list = []
        
        for each_key in one_dict:
            writerow_pa_list.append(one_dict[each_key])
        wr.writerow(writerow_pa_list)

In [7]:
async def get_article_body(article_url):
    async with aiohttp.ClientSession() as session:
        async with session.get(article_url, headers={'user-agent': 'Mozilla/5.0'}) as req:
            if req.status == 404:
                print('그런 페이지가 없다')
                return None # 없는 페이지라면 반환값은 None이 된다.
            html = await req.text()
            soup = BeautifulSoup(html, 'lxml')
            article_body = soup.find('div', {"itemprop" : "articleBody"}).get_text()
            return article_body

In [26]:
df_article_list_gen = pd.read_csv('seoulK2.csv', encoding='utf-8', chunksize=20000, dtype={'종목코드': object})

In [28]:
next(df_article_list_gen)

,'언론사명','회사명','종목코드','기사_URL','기사_업로드_날짜','기사_제목'
0,서울경제,한일건설,6440,https://www.sedaily.com/NewsView/1HW01D27QC,2012-02-27,"[리포트] GS글로벌, 자회사 디케이티 올 흑자전환 - 현대證"""
1,서울경제,한일건설,6440,https://www.sedaily.com/NewsView/1HOITWL5HX,2012-02-26,"표류 공모형 PF사업 정상화 나선다"""""
2,서울경제,한일건설,6440,https://www.sedaily.com/NewsView/1HLMGZHI6Q,2012-02-26,"[전국 패트롤] 인천亞게임 선수촌 건설 공사 내달 23일 첫 삽"""""
3,서울경제,한일건설,6440,https://www.sedaily.com/NewsView/1HTLKSF75G,2012-02-26,"대구 건설업계 기지개"""""
4,서울경제,한일건설,6440,https://www.sedaily.com/NewsView/1HPLA86YII,2012-02-26,"[분양하이라이트] 대우건설 광교 '푸르지오 월드마크' 청약"""""
...,...,...,...,...,...,...
19995,서울경제,경남은행,192520,https://www.sedaily.com/NewsView/1HQREO56G6,2015-06-01,"[오늘의 메모] 6월 2일"""""
19996,서울경제,경남은행,192520,https://www.sedaily.com/NewsView/1HNB5884XS,2015-06-01,"경남은행 ""BNK그룹 편입 효과 있네"""""
19997,서울경제,경남은행,192520,https://www.sedaily.com/NewsView/1HVU4838GO,2015-05-29,"[서울포럼 2015] 참석해주신 분들"""""
19998,서울경제,경남은행,192520,https://www.sedaily.com/NewsView/1HTK1DCKCQ,2015-05-26,"네이버페이 다음 달 25일 출시"""""


In [9]:
async def get_article_body_run(df_article_list_gen):
    try: # pickle 파일 있는지 없는지 확인
        with open('progress_getbody.pickle', 'rb') as f:
            last_progress_getbody = pickle.load(f)
    except FileNotFoundError:
        last_progress_getbody = [0,0]
        # 몇 번째 chunk인지, chunk 내에서 몇 번째인지 설정

    header = ['언론사명', '회사명', '종목코드', '기사_URL', '기사_업로드_날짜', '기사_제목', '기사_내용']
    
    progress_chunk = 0
    for each_chunk in df_article_list_gen:
        clear_output(wait=True)
        if last_progress_getbody[0] > progress_chunk:
            progress_chunk += 1
            continue
        
        idx = 0
        loop_check = 1
        progress_inside_chunk = 0
        
        while loop_check == 1:
            if last_progress_getbody[1] > progress_inside_chunk:
                progress_inside_chunk += 1
                idx += 1
                continue
            
            for _ in range(1,11):
                get_article_body_ins_list = []
                article_dict_list = []
                try:
                    article_dict = {}
                    for each_key in header:
                        if each_key != '기사_내용':
                            article_dict[each_key] = test_df.iloc[idx][each_key]
                        else:
                            article_dict[each_key] = None

                    article_dict_list.append(article_dict)
                    get_article_body_ins_list.append(get_article_body(article_dict['기사_URL']))
                    idx += 1
                except KeyError:
                    loop_check = 0
                    break

            body_bunch = await asyncio.gather(*get_article_body_ins_list)

            for idx, each in enumerate(article_dict_list):
                each['기사_내용'] = body_bunch[idx]
                save_to_csv_one_article(each, header)
                
            last_progress_getbody[1] = last_progress_getbody[1] + len(body_bunch)
            with open('progress_getbody.pickle', 'wb') as f:
                pickle.dump(last_progress_getbody, f)
            
            print(last_progress_getbody[0], last_progress_getbody[1])
            time.sleep(1)

        last_progress_getbody[0] += 1
        last_progress_getbody[1] = 0
        with open('progress_getbody.pickle', 'wb') as f:
            pickle.dump(last_progress_getbody, f)

In [10]:
await get_article_body_run(df_article_list_gen)

NameError: name 'test_df' is not defined

In [7]:
# 없애야 할 텍스트 목록
 # googletag.cmd.push(function() { googletag.display('div-gpt-ad-1567043459465-0'); });

In [3]:
with open('progress_getbody.pickle', 'rb') as f:
    last_progress_getbody = pickle.load(f)
    print(last_progress_getbody)

[0, 1530]


In [12]:
dd = None

In [24]:
with open('seoulK.csv', 'r', encoding='utf-8', newline='') as f:
    rdr= csv.reader(f)
    with open('seoulK2.csv', 'w', encoding='utf-8', newline='') as wf:
        wtr= csv.writer(wf)
        for r in rdr:
            wtr.writerow( (r[0], r[1], r[2], r[3], r[4], r[5]) )